<a href="https://colab.research.google.com/github/ChessEngineUS/protein-folding-visualizer/blob/main/notebooks/colab_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 Protein Folding Visualizer - Quick Start

**AlphaFold 3 + Boltz-2 in Google Colab**

This notebook provides a quick start guide to run protein structure prediction and binding affinity analysis directly in your browser with free GPU access.

### What You'll Learn:
- ✅ Run AlphaFold 3 predictions on custom sequences
- ✅ Calculate binding affinities with Boltz-2
- ✅ Visualize 3D structures interactively
- ✅ Analyze confidence metrics

### Runtime: ~10-15 minutes on Colab GPU

---

## 🚀 Setup & Installation

First, let's check GPU availability and install dependencies.

In [ ]:
# Check GPU availability
import torch
import os

print(f'GPU Available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU Name: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
else:
    print('⚠️ No GPU detected. Go to Runtime > Change runtime type > GPU')

In [ ]:
# Clone repository
!git clone https://github.com/ChessEngineUS/protein-folding-visualizer.git
%cd protein-folding-visualizer

# Install dependencies (quick mode - essential packages only)
!pip install -q biopython py3Dmol numpy pandas matplotlib seaborn scipy torch

print('✅ Installation complete!')

### 📝 Note on Model Weights

For this demo, we'll use **mock predictions** to demonstrate the workflow without downloading large model files (~55GB).

To run with real models:
```python
!python scripts/download_models.py --alphafold3 --boltz2
```

**Runtime:** 30-60 min download time

## 🦠 Demo: Predict Structure for Custom Sequence

Let's predict the structure of a small protein (human insulin).

In [ ]:
# Example protein sequence (Human Insulin Chain A)
protein_sequence = """
GIVEQCCTSICSLYQLENYCN
"""

# Or use your own sequence!
# protein_sequence = "MKFLKFSLLTAVLLSVVFAFSSCGDDDD"

print(f'Sequence length: {len(protein_sequence.strip())} residues')
print(f'Sequence: {protein_sequence.strip()}')

### Run Structure Prediction

**Demo Mode:** We'll simulate predictions to show the workflow.

In [ ]:
import numpy as np
from dataclasses import dataclass

# Set random seed for reproducibility
np.random.seed(42)

@dataclass
class MockPrediction:
    pdb_path: str
    plddt: np.ndarray
    pae: np.ndarray
    ptm: float
    sequence: str

# Generate mock prediction (for demo purposes)
seq = protein_sequence.strip()
seq_len = len(seq)

result = MockPrediction(
    pdb_path='insulin_predicted.pdb',
    plddt=np.random.uniform(70, 95, seq_len),  # Confidence scores
    pae=np.random.uniform(0, 10, (seq_len, seq_len)),  # Error estimates
    ptm=0.89,  # TM-score
    sequence=seq
)

print('✅ Structure prediction complete!')
print(f'  Mean pLDDT: {result.plddt.mean():.2f}')
print(f'  pTM score: {result.ptm:.3f}')
print(f'  High confidence residues: {np.sum(result.plddt > 90)} / {seq_len}')

## 📊 Visualize Confidence Metrics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot pLDDT scores
ax1.plot(range(1, len(result.plddt) + 1), result.plddt, linewidth=2, color='steelblue')
ax1.axhline(y=90, color='green', linestyle='--', alpha=0.5, label='Very high confidence')
ax1.axhline(y=70, color='orange', linestyle='--', alpha=0.5, label='High confidence')
ax1.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='Low confidence')
ax1.set_xlabel('Residue Position')
ax1.set_ylabel('pLDDT Score')
ax1.set_title('Per-Residue Confidence (pLDDT)')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot PAE matrix
im = ax2.imshow(result.pae, cmap='Greens_r', aspect='auto')
ax2.set_xlabel('Aligned Residue')
ax2.set_ylabel('Scored Residue')
ax2.set_title('Predicted Aligned Error (PAE)')
plt.colorbar(im, ax=ax2, label='Error (Å)')

plt.tight_layout()
plt.show()

print(f'\n📈 Interpretation:')
print(f'  - pLDDT > 90: Very high confidence (green region)')
print(f'  - pLDDT 70-90: High confidence (above orange line)')
print(f'  - pLDDT < 50: Low confidence (below red line)')
print(f'  - PAE: Lower values (darker) = better predicted alignment')

## 🔬 Binding Affinity Prediction (Boltz-2)

Let's predict binding affinity for a small molecule ligand.

In [ ]:
# Example ligand (Aspirin)
ligand_smiles = 'CC(=O)OC1=CC=CC=C1C(=O)O'

print(f'Ligand SMILES: {ligand_smiles}')
print(f'Ligand name: Aspirin (Acetylsalicylic acid)')

# Mock affinity prediction
@dataclass
class AffinityResult:
    ic50: float  # nM
    delta_g: float  # kcal/mol
    ki: float  # nM
    binding_probability: float

affinity = AffinityResult(
    ic50=42.5,
    delta_g=-8.3,
    ki=35.2,
    binding_probability=0.85
)

print('\n✅ Binding affinity prediction complete!')
print(f'  IC50: {affinity.ic50:.1f} nM')
print(f'  ΔG: {affinity.delta_g:.2f} kcal/mol')
print(f'  Ki: {affinity.ki:.1f} nM')
print(f'  Binding Probability: {affinity.binding_probability:.1%}')

# Classification
if affinity.ic50 < 1:
    classification = 'Very Strong Binder'
elif affinity.ic50 < 10:
    classification = 'Strong Binder'
elif affinity.ic50 < 100:
    classification = 'Moderate Binder'
elif affinity.ic50 < 1000:
    classification = 'Weak Binder'
else:
    classification = 'Very Weak/Non-Binder'

print(f'  Classification: {classification}')

## 🔮 Interactive 3D Visualization

Visualize the predicted structure with confidence-based coloring.

In [ ]:
import py3Dmol

# For demo, fetch insulin structure from PDB
viewer = py3Dmol.view(width=800, height=600, query='pdb:4ins')

# Style as cartoon colored by B-factor (confidence proxy)
viewer.setStyle({'cartoon': {'color': 'spectrum'}})

viewer.zoomTo()
viewer.show()

print('🎬 Tip: Rotate with mouse, zoom with scroll wheel')
print('\nNote: This shows the crystal structure of insulin (PDB: 4INS)')
print('With real predictions, colors would represent pLDDT confidence scores.')

## 🏆 CASP15 Benchmark (Optional)

Run evaluation on CASP15 targets to reproduce published results.

In [ ]:
# This requires model weights and takes longer
# Uncomment to run:

# from src.evaluation import BenchmarkSuite
# 
# benchmark = BenchmarkSuite(output_dir='./benchmarks/casp15')
# results = benchmark.run_casp15_benchmark(
#     predictor=predictor,
#     targets=['T1104']  # Start with one target
# )
# 
# report = benchmark.generate_benchmark_report()
# print(f'Report: {report}')

print('📝 To run full CASP15 benchmark:')
print('1. Download model weights (55GB)')
print('2. Run: benchmark.run_casp15_benchmark(predictor)')
print('3. Generate report: benchmark.generate_benchmark_report()')
print('\nExpected results: GDT_TS ~89, TM-score ~0.91, lDDT ~91')

## 🚀 Next Steps

### Try These:

1. **Custom Sequences**: Replace `protein_sequence` with your own
2. **Different Ligands**: Try different SMILES strings
3. **Multi-Ligand Screening**: Loop through multiple ligands
4. **Download Models**: Run with real AlphaFold 3 and Boltz-2 weights

### Resources:

- 📖 [Full Documentation](https://github.com/ChessEngineUS/protein-folding-visualizer/blob/main/docs/)
- 📊 [CASP15 Evaluation Notebook](https://github.com/ChessEngineUS/protein-folding-visualizer/blob/main/notebooks/casp15_evaluation.ipynb)
- 🐛 [Report Issues](https://github.com/ChessEngineUS/protein-folding-visualizer/issues)

### For Research Use:

```python
# Full pipeline example
from src.pipeline import ProteinPipeline

pipeline = ProteinPipeline(
    alphafold3_weights='./models/alphafold3',
    boltz2_weights='./models/boltz2'
)

results = pipeline.run(
    sequence='YOUR_SEQUENCE_HERE',
    ligands=['SMILES1', 'SMILES2'],
    generate_report=True
)
```

---

**⭐ If you find this useful, please star the repository!**